In [1]:
import random
import numpy as np
from collections import deque
import uuid

# -------------------------
# HCEPN: Hybrid Commitment + Emergent Personality Negotiation
# -------------------------

class CommitmentLedger:
    """
    Simple simulated ledger storing commitments and outcomes.
    Each commitment is immutable once recorded (simulated 'on-chain' entry).
    """
    def __init__(self):
        self.entries = []

    def record_commitment(self, agent_name, round_no, action, stake):
        entry = {
            "id": str(uuid.uuid4())[:8],
            "agent": agent_name,
            "round": round_no,
            "action": action,
            "stake": stake,
            "settled": False,
            "outcome": None
        }
        self.entries.append(entry)
        return entry

    def settle_entry(self, entry_id, outcome, penalty=0):
        for e in self.entries:
            if e["id"] == entry_id:
                e["settled"] = True
                e["outcome"] = outcome
                e["penalty"] = penalty
                return e
        return None

    def summary(self):
        return [e.copy() for e in self.entries]


class Agent:
    """
    Agent has:
    - a personality vector (affects baseline cooperation preference)
    - a policy parameter p_coop (probability baseline to cooperate)
    - reputation tokens (staked when committing)
    - short memory for recent outcomes
    - explain() that returns a natural-language reason for action
    """
    def __init__(self, name, personality_dim=3):
        self.name = name
        # personality vector values in [-1,1]: e.g., [trusting, risk_aversion, reciprocity]
        self.personality = np.clip(np.random.normal(0, 0.5, size=personality_dim), -1, 1)
        # baseline policy probability to cooperate (0..1)
        self.p_coop = float(np.clip(0.5 + 0.1 * self.personality[0], 0.05, 0.95))
        self.reputation = 10.0  # tokens you can stake
        self.score = 0.0
        self.memory = deque(maxlen=5)  # store (round, action, opponent_action, reward)
        # small mutation sigma used in evolutionary updates
        self.mutation_sigma = 0.05

    def decide_commit(self, round_no):
        """
        Decide whether to make a commitment in this round.
        Commitment probability influenced by trustworthiness trait and available reputation.
        """
        trust_trait = (self.personality[0] + 1) / 2  # 0..1
        if self.reputation <= 0:
            return None
        # higher trust trait -> more likely to offer commitment/stake
        if random.random() < 0.3 + 0.4 * trust_trait:
            # stake proportional to some function of reputation and risk_aversion
            risk_aversion = (self.personality[1] + 1) / 2  # 0..1
            stake = float(np.clip(self.reputation * (0.1 + 0.4*(1-risk_aversion)), 0.1, self.reputation))
            return {"round": round_no, "action": None, "stake": stake}
        return None

    def choose_action(self, opponent):
        """
        Action probability depends on:
          - baseline p_coop
          - reciprocity: increases cooperation prob if opponent cooperated recently
          - reputation pressure: if low reputation, more likely to cooperate to rebuild
        """
        baseline = self.p_coop
        reciprocity = 0.0
        if opponent.memory:
            # check last opponent action
            last = opponent.memory[-1]
            if last[1] == "cooperate":
                reciprocity += 0.15 * ((self.personality[2] + 1) / 2)  # reciprocity trait
            else:
                reciprocity -= 0.10 * ((1 - self.personality[2]) / 2)
        rep_pressure = 0.0
        if self.reputation < 3:
            rep_pressure += 0.2  # try to cooperate to rebuild reputation

        prob = float(np.clip(baseline + reciprocity + rep_pressure, 0.02, 0.98))
        action = "cooperate" if random.random() < prob else "compete"
        return action, prob

    def record_outcome(self, round_no, action, opp_action, reward):
        self.memory.append((round_no, action, opp_action, reward))
        self.score += reward

    def explain(self, action, prob, stake=None):
        """
        Return a short explanation that mixes personality and recent context.
        """
        parts = []
        parts.append(f"I acted '{action}' (p={prob:.2f})")
        if stake:
            parts.append(f"and staked {stake:.2f} reputation tokens to show commitment.")
        # personality-driven reason
        trust = self.personality[0]
        reciprocity = self.personality[2]
        if action == "cooperate":
            if reciprocity > 0:
                parts.append("I expected reciprocity from past cooperation.")
            elif trust > 0:
                parts.append("my trusting trait guided cooperation.")
            else:
                parts.append("I chose cooperation to avoid longer-term penalties.")
        else:
            if trust < -0.2:
                parts.append("my low-trust trait made me avoid cooperating.")
            else:
                parts.append("I exploited a high reward opportunity this round.")
        # short memory hint
        if self.memory:
            last = self.memory[-1]
            parts.append(f"Last round I got {last[3]} reward.")
        return " ".join(parts)

    def evolutionary_update(self, keep_factor=0.7):
        """
        Mutate p_coop and personality slightly based on recent rewards.
        If recent average reward was high → keep policy more; if low → mutate more.
        """
        if len(self.memory) == 0:
            return
        avg_reward = np.mean([m[3] for m in self.memory])
        # adapt mutation amplitude inversely to performance
        adapt = float(np.clip(1.0 - (avg_reward+1)/6.0, 0.1, 2.0))
        sigma = self.mutation_sigma * adapt
        # mutate personality a bit
        self.personality += np.random.normal(0, sigma, size=self.personality.shape)
        self.personality = np.clip(self.personality, -1, 1)
        # mutate p_coop a bit toward mean of personality[0]
        target = float(np.clip(0.5 + 0.1*self.personality[0], 0.02, 0.98))
        # keep_factor controls how much of old p_coop remains
        self.p_coop = float(np.clip(self.p_coop*keep_factor + target*(1-keep_factor) + np.random.normal(0, sigma), 0.01, 0.99))
        # small drift in mutation_sigma
        self.mutation_sigma = float(np.clip(self.mutation_sigma * (1 + np.random.normal(0, 0.01)), 0.01, 0.2))

    def penalize_for_breach(self, penalty):
        self.reputation = max(0.0, self.reputation - penalty)

    def reward_reputation(self, amount):
        self.reputation += amount

# Simple payoff matrix (same as original)
def payoff(a1, a2):
    if a1 == "cooperate" and a2 == "cooperate":
        return 3.0, 3.0
    elif a1 == "cooperate" and a2 == "compete":
        return 0.0, 5.0
    elif a1 == "compete" and a2 == "cooperate":
        return 5.0, 0.0
    else:
        return -1.0, -1.0

# ---------- Simulation ----------
def run_simulation(rounds=20, commit_enabled=True, settle_penalty_multiplier=1.0, evolve_every=5):
    ledger = CommitmentLedger()
    a = Agent("Claude-You")
    b = Agent("Gemini-Other")

    print("Initial state:")
    print(f"{a.name}: p_coop={a.p_coop:.2f}, personality={a.personality}, reputation={a.reputation:.2f}")
    print(f"{b.name}: p_coop={b.p_coop:.2f}, personality={b.personality}, reputation={b.reputation:.2f}\n")

    for r in range(1, rounds+1):
        print(f"--- Round {r} ---")
        # each agent optionally creates a commitment (stake)
        commit_a = a.decide_commit(r) if commit_enabled else None
        commit_b = b.decide_commit(r) if commit_enabled else None

        if commit_a:
            commit_a["action"] = None
            entry_a = ledger.record_commitment(a.name, r, None, commit_a["stake"])
            a.reputation -= commit_a["stake"]  # temp lock of reputation
            commit_a["entry_id"] = entry_a["id"]
            print(f"{a.name} created commitment (id={entry_a['id']}) staking {commit_a['stake']:.2f}")

        if commit_b:
            commit_b["action"] = None
            entry_b = ledger.record_commitment(b.name, r, None, commit_b["stake"])
            b.reputation -= commit_b["stake"]
            commit_b["entry_id"] = entry_b["id"]
            print(f"{b.name} created commitment (id={entry_b['id']}) staking {commit_b['stake']:.2f}")

        # choose actions (they don't have to follow commitments, but ledger will record breach)
        action_a, prob_a = a.choose_action(b)
        action_b, prob_b = b.choose_action(a)

        # settle commitments: if agent committed and action != declared (here we didn't declare explicit promised action),
        # we interpret a commitment as a "promise to cooperate" (design choice)
        if commit_a:
            expected = "cooperate"
            if action_a != expected:
                # breach: heavy penalty proportional to stake
                penalty = commit_a["stake"] * settle_penalty_multiplier
                ledger.settle_entry(commit_a["entry_id"], "breach", penalty=penalty)
                a.penalize_for_breach(penalty)
                print(f"COMMITMENT BREACH: {a.name} breached commitment {commit_a['entry_id']} -> penalty {penalty:.2f}")
            else:
                ledger.settle_entry(commit_a["entry_id"], "honored", penalty=0)
                a.reward_reputation(commit_a["stake"]*0.2)  # reward for honoring
                print(f"{a.name} honored commitment {commit_a['entry_id']} and regained trust.")

        if commit_b:
            expected = "cooperate"
            if action_b != expected:
                penalty = commit_b["stake"] * settle_penalty_multiplier
                ledger.settle_entry(commit_b["entry_id"], "breach", penalty=penalty)
                b.penalize_for_breach(penalty)
                print(f"COMMITMENT BREACH: {b.name} breached commitment {commit_b['entry_id']} -> penalty {penalty:.2f}")
            else:
                ledger.settle_entry(commit_b["entry_id"], "honored", penalty=0)
                b.reward_reputation(commit_b["stake"]*0.2)
                print(f"{b.name} honored commitment {commit_b['entry_id']} and regained trust.")

        # compute payoffs
        r_a, r_b = payoff(action_a, action_b)

        # incorporate reputation effect into reward (agents with better reputation get small bonus)
        r_a += 0.05 * (a.reputation/10.0)
        r_b += 0.05 * (b.reputation/10.0)

        a.record_outcome(r, action_a, action_b, r_a)
        b.record_outcome(r, action_b, action_a, r_b)

        # provide explanations
        explanation_a = a.explain(action_a, prob_a, stake=commit_a["stake"] if commit_a else None)
        explanation_b = b.explain(action_b, prob_b, stake=commit_b["stake"] if commit_b else None)
        print(f"{a.name} -> {action_a} | explanation: {explanation_a}")
        print(f"{b.name} -> {action_b} | explanation: {explanation_b}")
        print(f"Round rewards: {a.name}={r_a:.2f}, {b.name}={r_b:.2f}")
        print(f"Current reputation: {a.name}={a.reputation:.2f}, {b.name}={b.reputation:.2f}\n")

        # evolutionary updates periodically
        if r % evolve_every == 0:
            print(">>> Evolutionary update phase (mutating policies/personality based on recent outcomes)")
            a.evolutionary_update()
            b.evolutionary_update()
            print(f"After evolution: {a.name}: p_coop={a.p_coop:.2f}, personality={np.round(a.personality,3)}, mutation_sigma={a.mutation_sigma:.3f}")
            print(f"After evolution: {b.name}: p_coop={b.p_coop:.2f}, personality={np.round(b.personality,3)}, mutation_sigma={b.mutation_sigma:.3f}\n")

    print("=== Simulation complete ===")
    print(f"Final scores: {a.name}={a.score:.2f}, {b.name}={b.score:.2f}")
    print("Ledger summary:")
    for e in ledger.summary():
        print(e)

if __name__ == "__main__":
    # run demo
    run_simulation(rounds=20, commit_enabled=True, settle_penalty_multiplier=1.5, evolve_every=5)


Initial state:
Claude-You: p_coop=0.58, personality=[0.79848054 0.37541148 0.54714301], reputation=10.00
Gemini-Other: p_coop=0.55, personality=[ 0.54740819 -1.         -0.08341875], reputation=10.00

--- Round 1 ---
Claude-You -> compete | explanation: I acted 'compete' (p=0.58) I exploited a high reward opportunity this round. Last round I got 5.05 reward.
Gemini-Other -> cooperate | explanation: I acted 'cooperate' (p=0.55) my trusting trait guided cooperation. Last round I got 0.05 reward.
Round rewards: Claude-You=5.05, Gemini-Other=0.05
Current reputation: Claude-You=10.00, Gemini-Other=10.00

--- Round 2 ---
Claude-You created commitment (id=5fd3bd53) staking 2.25
Gemini-Other created commitment (id=732efadf) staking 5.00
Claude-You honored commitment 5fd3bd53 and regained trust.
COMMITMENT BREACH: Gemini-Other breached commitment 732efadf -> penalty 7.50
Claude-You -> cooperate | explanation: I acted 'cooperate' (p=0.70) and staked 2.25 reputation tokens to show commitment. I e